# Active Learning

Para que la generalización de las características aprendidas por la red neuronal durante su entrenamiento sea buena, el conjunto de entrenamiento debe contener una gran cantidad de muestras representativas de la tarea a resolver, pues es a través de éstas y de la función de pérdida que el modelo actualiza los cientos de miles o millones de parámetros presentes en éste. De esta manera, el rendimiento del modelo depende también de la cantidad y calidad de muestras presentes en el conjunto de entrenamiento. Sin embargo, la construcción de un conjunto de entrenamiento de calidad puede ser un proceso costoso y tedioso, pues requiere de la etiquetación manual de un gran número de muestras. En este sentido, la adquisición de datos se vuelve un proceso crítico en el desarrollo de cualquier tipo de modelo de inteligencia artificial.

Una técnica especialmente útil para añadir un conjunto de datos etiquetados que reduce de forma óptima la cantidad de muestras nuevas a etiquetar basado en la información relevante para la tarea a resolver se conoce como Active Learning o Deep Active Learning.

De acuerdo con *Ren, P., et. al. (Survey of Deep Active Learning)*, active learning es un método dedicado a estudiar cómo obtener el mayor número posible de ganancias de rendimiento etiquetando el menor número posible de muestras. Más concretamente, su objetivo es seleccionar las muestras más útiles del conjunto de datos sin etiquetar y entregárselas a un ente llamado oráculo (por ejemplo un anotador humano) para que las etiquete, a fin de reducir el coste del etiquetado lo máximo posible, manteniendo el rendimiento.

El procedimiento para aplicar la técnica de active learning es completamente iterativo de la siguiente forma: El oráculo es consultado con las muestras que la red considera más informativas, dada su configuración actual. Una vez asignadas las etiquetas de dichas muestras, el modelo es entrenado con el conjunto original y este nuevo conjunto etiquetado, para finalmente repetir este proceso hasta que el modelo muestre un rendimiento sólido o se cumpla una cierta condición.

El componente más importante de esta técnica es la estrategia de consulta, cuyas variaciones proveen de diferentes algoritmos de active learning. La estrategia de consulta empleada para extender las muestras de entrenamiento para el modelo de xenofobia fue basado en el modelo, en la cual los casos de consulta son seleccionados basados en una medida producida por el modelo dada una muestra o instancia. En particular, se usó la estrategia *Least Confidence* que puede resumirse en:
* Se crea un clasificador inicial.
* Se aplica el clasificador actual a cada muestra no etiquetada.
* Se extraen y etiquetan $n$ muestras tales que el clasificador está menos seguro de la pertenencia a las clases (probabilidad de pertenencia a ambas clases es cercana a $0.5$).
* Se entrena un nuevo clasificador con todos los datos etiquetados.

En este notebook se muestra el proceso de extensión de muestras de entrenamiento para el modelo de xenofobia usando la técnica de active learning a través del método least confidence.

In [1]:
#Imports

#HuggingFace library
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, TextClassificationPipeline
from datasets import Dataset, Value, ClassLabel, Features

#PyTorch Neural Networks
import torch
import torch.nn as nn

#data reading
import pandas as pd

#math
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px

### Carga del modelo de xenofobia y de los datos

Se cargará el modelo de xenofobia entrenado en el notebook xenofobia.ipynb, y los datos. Tal y como se describió en dicho notebook.

In [2]:
model_name = './checkpoint-5250'

#load model from model name using huggingface library
model = AutoModelForSequenceClassification.from_pretrained(
        model_name, return_dict=True, num_labels=2)

#load tokenizer and config it (based on robertuito github)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128

In [3]:
active_data_df = pd.read_csv('./assets/data/activelearning.csv')
active_data_df.sample(5)

,id,text
1251,1220683909740090112,@usuario Quien entra aún país ajeno a la fuerz...
110,1245938019523160064,@usuario @usuario El problema de los venezolan...
1330,1123895305894800000,@usuario @usuario Mi opinion desde aqui para e...
114,1361427681091470080,"@usuario De ahi, estamos a un paso de que la O..."
801,1363194081657179904,"CHILE ESTADO CORRUPTO, POLICÍA SINIESTRA Y DEL..."


In [4]:
def tokenize(batch):
        """Tokenize text in current mini batch. This is a util function for get_dataset_from_dataframes function

        Args:
            batch (batched datasets.arrow_dataset.Dataset)
        
        Returns:
            [datasets.arrow_dataset.Dataset]: Mapped text-label dataset
        """
        return tokenizer(batch['text'], padding=False, truncation=True)

def format_dataset(dataset):
    """Map text-label for specific dataset from pandas. This is a util function for get_dataset_from_dataframes function

    Args:
        dataset (datasets.arrow_dataset.Dataset): Dataset from pandas DataFrame

    Returns:
        [datasets.arrow_dataset.Dataset]: Mapped text-label dataset
    """
    def get_labels(examples):
        return {'labels': examples['label']}

    dataset = dataset.map(get_labels)
    return dataset

#Features to map insto dataset
features = Features({
    'text': Value('string'),
    })
#create dataset from pandas dataframe for model consumption
active_data = Dataset.from_pandas(active_data_df, features=features)
active_data = active_data.map(tokenize, batched=True, batch_size=8)

#to be able to use batched training, we need to use a data collator
data_collator = DataCollatorWithPadding(tokenizer, padding='longest')

  0%|          | 0/175 [00:00<?, ?ba/s]

### Inferencia de los datos
debe instanciarse una clase Trainer con el modelo entrenado y el tokenizador como argumentos. La predicción se realiza a través del método predict, el cual recibe como argumento un texto y devuelve la probabilidad de pertenencia a cada una de las clases.

In [5]:
trainer_args = {
        "model": model,
        "data_collator": data_collator,
        "tokenizer": tokenizer}

test_trainer = Trainer(**trainer_args)
raw_pred= test_trainer.predict(active_data)
y_pred = np.argmax(raw_pred.predictions, axis=-1)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1394
  Batch size = 8


  0%|          | 0/175 [00:00<?, ?it/s]

Las probabilidades de pertenencia a cada una de las clases de cada texto regresado por el método predict no se encuentran normalizados. Se muestra el siguiente ejemplo:

<figure>
    <img src="./assets/images/logits.png"
         alt="Logits"
         style="max-width: 80%; height: auto">
    <figcaption>Logits producidos por el modelo de seis textos</figcaption>
</figure>


a estos valores no normalizados se les conoce como logits. Para obtener las probabilidades de pertenencia a cada una de las clases, se aplica la función softmax a los logits. La función softmax es una función de activación que se utiliza para normalizar los logits de tal manera que la suma de las probabilidades de pertenencia a cada una de las clases sea igual a $1$. La función softmax se define con la siguiente ecuación:
$$\sigma (z)_{j} = \frac{exp(z_{j})}{\sum^{n}_{i=1}exp(z_{k})}$$

Con $exp(\cdot)$ la funcion exponencial, $n$ el número de clases y $z_{j}$ el valor del logit de la clase $j$.

La siguiente celda de código normaliza los logits de los textos y construye un dataframe con las probabilidades de cada clase en una columna por separado.

In [18]:
softmax = nn.Softmax(dim=1)
probas = softmax(torch.tensor(raw_pred.predictions)).numpy()
probas = pd.DataFrame(probas, columns=['Prob_class0', 'Prob_class1'])

Para extraer las muestras tales que el clasificador está menos seguro de la pertenencia a las clases, se restan las probabilidades de clase de cada texto. Esto nos dirá el grado de "confusión" por muestra. Como ejemplo consideremos los siguientes casos:
* Dado un texto A, el modelo predice las probabilidades de pertenencia a las clases de $0.9$ y $0.1$. Para la clase no xenófoba y xenófoba respectivamente, la diferencia entre ambas probabilidades es de $0.8$. Este caso es el más claro de todos, ya que el modelo está muy seguro de la pertenencia a la clase no xenófoba.
* Dado un texto B, el modelo predice las probabilidades de pertenencia a las clases de $0.49$ y $0.51$. Para la clase no xenófoba y xenófoba respectivamente, la diferencia entre ambas probabilidades es de $0.01$. Este caso es el más confuso de todos, ya que el modelo no está seguro a qué clase pertenece el texto B.

Con esto en mente, sabemos que las muestras más representativas serán aquellas que tengan una diferencia entre las probabilidades más pequeña. Sin embargo, el modelo, ni el método, ofrecen una idea de que tan pequeña debe ser la diferencia para considerar que el modelo está menos seguro de la pertenencia a las clases. Este valor umbral debe definirse de manera empírica, basado en la cantidad de muestras que se desean etiquetar.

La siguiente celda realiza la resta entre las probabilidades, para así obtener las muestras más representativas, así mismo se define la etiqueta designada por el modelo actual y el texto analizado.

In [39]:
#construct dataframe with predictions, probabilities and text
probas['margin_sampling'] = (probas.Prob_class0 - probas.Prob_class1).abs()
probas['id'] = active_data_df['id']
probas['text'] = active_data_df['text']
probas['predicted_label'] = np.argmax(raw_pred.predictions, axis=-1)
probas.predicted_label = probas.predicted_label.replace({0:'ok', 1:'xenófobo'})
probas.sample(5)

,Prob_class0,Prob_class1,margin_sampling,id,text,predicted_label
322,0.999943,0.000057,0.999887,1096158206349459968,@usuario @usuario le pregunto a la @usuario po...,ok
1106,0.000087,0.999913,0.999826,1358250094399810048,@usuario En vez de sacarlos cada día se la pon...,xenófobo
978,0.999775,0.000225,0.999549,1299872225542249984,emoji luces de policía emoji emoji luces de po...,ok
526,0.000272,0.999728,0.999456,1155687859640320000,"La Guajira, no resiste más migrantes venezolan...",xenófobo
636,0.000197,0.999803,0.999606,1326142115114589952,CHILE NECESITA ABOGADOS POR LA VERDAD 30 AÑOS ...,xenófobo


De manera visual se presenta una gráfica de puntos. Cada punto pertenece a un texto, sus coordenadas $x$ e $y$ corresponden al valor de la probabilidad de pertenencia a las clases no xenófobo y xenófobo respectivamente. En dicha gráfica se muestra un cuadro color naranja que encierra las muestras más representativas.

Debido a la contrucción de este gráfico, los puntos siempre se encontrarán sobre la recta $y = -x$

Note que los extremos, izquierdo y derecho contendrán textos para los cuales el modelo es más confidente de la pertenencia a una clase. Esto se debe a que la diferencia entre las probabilidades de pertenencia a las clases es cercana a $1.0$.

In [36]:
#plot scatter of probabilities given by the model from each text
fig = px.scatter(probas, x="Prob_class0", y="Prob_class1", color="predicted_label",
                hover_data=['text'], width=1200, height=800)
fig.add_hline(y=0.5, line_dash="dash")
fig.add_vline(x=0.5, line_dash="dash")
fig.add_vrect(x0=0.3, x1=0.7, y0=0.3,y1=0.7, fillcolor="LightSalmon", opacity=0.4, line_width=1)
fig.show()

Una vez construido el conjunto de datos, se define un umbral para la diferencia entre las probabilidades de valor $0.3$. Con esto se obtienen las muestras más representativas, las cuales se etiquetan manualmente.

In [40]:
to_label = probas[probas.margin_sampling<=0.3]
to_label.sample(5)

,Prob_class0,Prob_class1,margin_sampling,id,text,predicted_label
1168,0.434580,0.565420,0.130840,1179940011019569920,@usuario Cooño pero que tipo para bruto!! Jjaj...,xenófobo
669,0.439290,0.560710,0.121420,1285187904227229952,La inmigración nunca va a parar ! Lo que se pu...,xenófobo
1134,0.403014,0.596986,0.193973,1084918484042240000,@usuario @usuario Está avanzando un proceso de...,xenófobo
381,0.508840,0.491160,0.017680,1321891553976219904,@usuario Esto hay que pararlo. Vienen aquí sin...,ok
652,0.501519,0.498481,0.003038,1239560358802619904,"@usuario Mpdo por ti estamos así, podías cerra...",ok
